<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/holster/VIT_quickdraw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Helpful links

# https://github.com/benisalla/Tiny-ViT-Transformer-from-scratch?tab=readme-ov-file#training
# https://github.com/benisalla/Tiny-ViT-Transformer-from-scratch?tab=readme-ov-file#training
# https://medium.com/@tyler_yu/vit-from-scratch-61debb718e99

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class VisionEmbedding(nn.Module):
    def __init__(self, config: vit_config):
        super().__init__()

        self.config  = config
        self.patch_embedding = nn.Sequential(
            nn.Conv2d(
                in_channels=config.num_channels,
                out_channels=config.embd_dim,
                kernel_size=config.patch_size,
                stride=config.patch_size,
                padding="valid"
            ),
            nn.Flatten(start_dim=2)
        )

        self.cls_token = nn.Parameter(torch.randn(size=(1, 1, config.embd_dim)), requires_grad=True)
        self.pos_embeddings = nn.Parameter(torch.randn(size=(1, config.num_patches + 1, config.embd_dim)), requires_grad=True)
        self.dropout = nn.Dropout(p=config.dropout)

    def forward(self, x : torch.Tensor) -> torch.Tensor:
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)

        patch_embd = self.patch_embedding(x).transpose(2,1)
        patch_embd = torch.cat([cls_token, patch_embd], dim=1)
        embd = self.pos_embeddings + patch_embd
        embd = self.dropout(embd)
        return embd

In [ ]:
class AttentionHead(nn.Module):

  def __init__(self, embed_dim, head_size):
    super().__init__()
    ...

  def forward(self, x):
    ...



In [ ]:
class FeedForward(nn.Module):

  def __init__(self):
    super().__init__()
    ...

  def forward(self, x):
    ...

In [ ]:
class MultiHeadAttention(nn.Module):

  def __init__(self):
    super().__init__()
    ...

  def forward(self, x):
    ...

In [ ]:
class AttentionBlock(nn.Module):

  def __init__(self):
    super().__init__()
    ...

  def forward(self, x):
    ...

In [ ]:
class VisionTransformer(nn.Module):

  def __init__(self):
    ...

  def forward(self):
    ...